# Documentación en Notebook (.ipynb)
## Métodos numéricos aplicados a filtros faciales (MediaPipe Face Mesh) y detección de bordes (OpenCV/Sobel)

**Materia:** Visión por Computadora  
**Proyecto:** Visión por Computadora Web con MediaPipe y OpenCV.js  

**Objetivo del notebook:** Explicar, con demostraciones matemáticas básicas, cómo se usan métodos numéricos en:
- Conversión a escala de grises (combinación lineal / aproximación numérica).
- Filtrado por convolución (máscaras y sumas ponderadas).
- Derivadas aproximadas por **diferencias finitas** (Sobel).
- Umbralización (función por partes).
- Transformaciones geométricas para superponer filtros (escalamiento + traslación usando landmarks).
- Suavizado simple (promedio móvil) para estabilizar puntos (opcional, pero muy usado).

> Nota: Aunque el proyecto final corre en **JavaScript** con **MediaPipe** y **OpenCV.js**, aquí se muestran las matemáticas con **Python + NumPy** para entender el fundamento numérico.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Utilidad para mostrar imágenes (grises o RGB) de forma simple
def show(img, title=None):
    plt.figure()
    if img.ndim == 2:
        plt.imshow(img, cmap="gray")
    else:
        plt.imshow(img)
    if title:
        plt.title(title)
    plt.axis("off")
    plt.show()

# Generamos una imagen sintética para demostrar bordes y filtros (cuadro blanco en fondo negro)
H, W = 240, 320
img = np.zeros((H, W), dtype=np.float32)
img[70:170, 110:210] = 1.0  # rectángulo blanco
show(img, "Imagen sintética (escala 0..1)")

## 1) Escala de grises como combinación lineal (método numérico)
En la web, cuando capturas un frame (RGB), la conversión a gris se hace con una **combinación lineal** de canales:

\[
Y \approx 0.299R + 0.587G + 0.114B
\]

Esto es una **aproximación numérica** del brillo percibido (luma). En términos de método numérico:
- Es un **promedio ponderado** (suma de productos).
- Reduce dimensionalidad de 3 canales a 1 canal, conservando información útil para bordes.

Abajo se demuestra con una imagen RGB sintética.


In [ ]:
# Creamos una imagen RGB sintética: gradiente + un cuadrado
rgb = np.zeros((H, W, 3), dtype=np.float32)
x = np.linspace(0, 1, W, dtype=np.float32)
rgb[..., 0] = x          # R gradiente
rgb[..., 1] = x[::-1]    # G gradiente inverso
rgb[..., 2] = 0.2        # B constante
rgb[70:170, 110:210, :] = 1.0  # cuadrado blanco

# Conversión a gris (luma)
gray = 0.299*rgb[...,0] + 0.587*rgb[...,1] + 0.114*rgb[...,2]

show(rgb, "RGB sintética")
show(gray, "Escala de grises (luma)")

# Demostración numérica básica: valor de gris en un pixel
i, j = 100, 150
R, G, B = rgb[i, j]
Y = gray[i, j]
print("Pixel (i,j) =", (i,j))
print("R,G,B =", float(R), float(G), float(B))
print("Y = 0.299R + 0.587G + 0.114B =", float(Y))

## 2) Filtrado por convolución (sumas ponderadas en una vecindad)
Muchos filtros clásicos (blur, sharpen, Sobel, etc.) se expresan como **convoluciones**:

\[
(I * K)(x,y) = \sum_{u}\sum_{v} I(x-u, y-v)\,K(u,v)
\]

Esto es un método numérico porque:
- Aproxima operaciones continuas (integrales) mediante **sumas discretas**.
- Aplica un operador local usando una ventana (kernel).

A continuación implementamos una convolución 2D “a mano” para mostrar la idea.


In [ ]:
def conv2d(image, kernel):
    # Convolución 2D básica (padding con ceros)
    kh, kw = kernel.shape
    ph, pw = kh//2, kw//2
    padded = np.pad(image, ((ph, ph), (pw, pw)), mode="constant")
    out = np.zeros_like(image, dtype=np.float32)
    # Método directo O(H*W*kh*kw)
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            region = padded[y:y+kh, x:x+kw]
            out[y, x] = np.sum(region * kernel)
    return out

# Kernel de desenfoque (promedio) 3x3
K_blur = np.ones((3,3), dtype=np.float32) / 9.0
blurred = conv2d(img, K_blur)

show(img, "Original (gris)")
show(blurred, "Blur por convolución 3x3")

# Demostración numérica: cálculo del pixel cercano a un borde
y, x = 70, 110  # esquina del rectángulo (cambio brusco)
region = np.pad(img, ((1,1),(1,1)))[y:y+3, x:x+3]
print("Región 3x3 alrededor del pixel (70,110):\n", region)
print("Blur = suma(region * K_blur) =", float(np.sum(region * K_blur)))

## 3) Sobel = derivadas aproximadas por diferencias finitas
Un borde puede detectarse midiendo el **gradiente** (cambio de intensidad). En continuo:

\[
\nabla I = \left(\frac{\partial I}{\partial x},\frac{\partial I}{\partial y}\right)
\]

En una imagen discreta, las derivadas se aproximan con **diferencias finitas**.
Sobel usa dos kernels (máscaras) que combinan diferencia y suavizado.

Kernels Sobel clásicos:

\[
G_x=\begin{bmatrix}-1&0&1\\-2&0&2\\-1&0&1\end{bmatrix},\quad
G_y=\begin{bmatrix}-1&-2&-1\\0&0&0\\1&2&1\end{bmatrix}
\]

Magnitud del borde:

\[
|G| \approx \sqrt{G_x^2 + G_y^2}
\]


In [ ]:
Gx = np.array([[-1,0,1],
               [-2,0,2],
               [-1,0,1]], dtype=np.float32)
Gy = np.array([[-1,-2,-1],
               [ 0, 0, 0],
               [ 1, 2, 1]], dtype=np.float32)

Ix = conv2d(img, Gx)
Iy = conv2d(img, Gy)
mag = np.sqrt(Ix**2 + Iy**2)

# Normalizamos para visualizar
mag_vis = mag / (mag.max() + 1e-8)

show(Ix, "Sobel: gradiente en X (Ix)")
show(Iy, "Sobel: gradiente en Y (Iy)")
show(mag_vis, "Magnitud del gradiente |G| (normalizada)")

print("max(|G|) =", float(mag.max()))

## 4) Umbralización (threshold) como función por partes
Después de calcular \(|G|\), se aplica un umbral para “limpiar” ruido:

\[
B(x,y)=\begin{cases}
1 & |G(x,y)| \ge T\\
0 & |G(x,y)| < T
\end{cases}
\]


In [ ]:
T = 0.35  # umbral (en magnitud normalizada)
edges = (mag_vis >= T).astype(np.float32)

show(mag_vis, "Magnitud |G| (normalizada)")
show(edges, f"Bordes binarios con umbral T={T}")

pct = 100.0 * edges.mean()
print(f"Porcentaje de pixeles detectados como borde: {pct:.2f}%")

## 5) Landmarks y superposición de filtros: escalamiento + traslación
MediaPipe Face Mesh devuelve landmarks en coordenadas **normalizadas** (0..1).  
Para dibujar sobre un `canvas` de tamaño \(W\times H\):

\[
x_{px} = x_{norm}\cdot W,\quad y_{px} = y_{norm}\cdot H
\]

### Tamaño del filtro (geometría numérica)
Para escalar un filtro (ej. bigote), se usa distancia euclidiana entre dos landmarks (comisuras de la boca):

\[
d = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}
\]

Luego:

\[
w_{filtro} = s\cdot d
\]

donde \(s\) es un factor de ajuste.


In [ ]:
# Simulamos landmarks normalizados (0..1) en un frame de webcam
Wc, Hc = 640, 480
mouth_left  = np.array([0.42, 0.60])
mouth_right = np.array([0.58, 0.60])
nose_tip    = np.array([0.50, 0.52])

def to_px(p_norm, W, H):
    return np.array([p_norm[0]*W, p_norm[1]*H], dtype=np.float32)

ml = to_px(mouth_left,  Wc, Hc)
mr = to_px(mouth_right, Wc, Hc)
nt = to_px(nose_tip,    Wc, Hc)

d = np.linalg.norm(mr - ml)  # distancia boca

s = 1.25
mustache_w = s * d
mustache_h = 0.35 * mustache_w  # proporción

center = (ml + mr) / 2.0
offset_up = 0.20 * mustache_h
top_left = center - np.array([mustache_w/2, mustache_h/2 + offset_up], dtype=np.float32)

print("Boca izq px =", ml)
print("Boca der px =", mr)
print("Distancia boca d =", float(d))
print("Bigote (w,h) =", (float(mustache_w), float(mustache_h)))
print("Top-left bigote =", top_left)

# Visualización simple (puntos y rectángulo del filtro)
canvas = np.zeros((Hc, Wc), dtype=np.float32)

def mark(img, p, val=1.0, r=4):
    x,y = int(p[0]), int(p[1])
    img[max(0,y-r):min(img.shape[0],y+r), max(0,x-r):min(img.shape[1],x+r)] = val

mark(canvas, ml, 0.7)
mark(canvas, mr, 0.7)
mark(canvas, nt, 0.9)

x0, y0 = int(top_left[0]), int(top_left[1])
x1, y1 = int(top_left[0] + mustache_w), int(top_left[1] + mustache_h)
canvas[max(0,y0):min(Hc,y1), max(0,x0):min(Wc,x1)] = np.maximum(
    canvas[max(0,y0):min(Hc,y1), max(0,x0):min(Wc,x1)], 0.3
)

show(canvas, "Simulación: landmarks y caja del bigote (aprox.)")

## 6) Suavizado de puntos (promedio móvil / suavizado exponencial)
Para evitar que el filtro “tiemble” por ruido en landmarks, se usa un suavizado:

\[
p_t^{suave} = \alpha\,p_t + (1-\alpha)\,p_{t-1}^{suave}
\]

Es un filtro numérico simple (IIR de primer orden).


In [ ]:
np.random.seed(7)
t = np.arange(200)
true = 0.5 + 0.05*np.sin(2*np.pi*t/50)
noisy = true + 0.02*np.random.randn(len(t))

alpha = 0.25
smooth = np.zeros_like(noisy)
smooth[0] = noisy[0]
for i in range(1, len(t)):
    smooth[i] = alpha*noisy[i] + (1-alpha)*smooth[i-1]

plt.figure()
plt.plot(t, true, label="Real")
plt.plot(t, noisy, label="Con ruido")
plt.plot(t, smooth, label=f"Suavizada (alpha={alpha})")
plt.legend()
plt.title("Suavizado exponencial de un landmark (demostración)")
plt.xlabel("Frame")
plt.ylabel("Posición normalizada")
plt.show()

print("Ejemplo frame 10 (noisy, smooth) =", float(noisy[10]), float(smooth[10]))

## 7) Relación directa con tu implementación Web (JS)
En tu proyecto (HTML/JS):

- **Escala de grises**: promedio ponderado (o `cv.cvtColor` en OpenCV.js).
- **Sobel**: kernels \(G_x\) y \(G_y\) (con `cv.Sobel`) → convolución + diferencias finitas.
- **Umbral**: comparación por partes (con `cv.threshold` o lógica propia).
- **Landmarks**: MediaPipe entrega coordenadas normalizadas → a píxeles multiplicando por ancho/alto del canvas.
- **Escalamiento del filtro**: distancia euclidiana entre landmarks.
- **Suavizado**: (opcional) para estabilidad visual.

### Pseudocódigo equivalente
```text
x_px = x_norm * canvasWidth
y_px = y_norm * canvasHeight

d = sqrt((x2-x1)^2 + (y2-y1)^2)
w_filter = s * d

Ix = conv2(gray, Gx)
Iy = conv2(gray, Gy)
mag = sqrt(Ix^2 + Iy^2)
edges = mag >= T
```
